Классификация текстов при помощи метода K ближайших соседей

Постановка задачи: разработать программу для классификации текстов при помощи метода K ближайших соседей
Для этого нам нужно:
- Разработать функционал извлечения текстовых данных из файлов, отбрасывая стоп-слова для русского языка (стоп-слова - это слова, которые довольно часто встречаются в разговорной речи и письме, но при этом не несут никакой смысловой нагрузки, а следовательно не могут быть маркерами для текстов определенного класса)
- Разработать функционал для составления списка наиболее часто встречающихся слов для каждого класса текстов
- Разработать функционал для составления нормализованного вектора для каждого текста и его дальнейшего включения в датасет
- Разработать функционал для обеспечения функционирования непосредственно метода K ближайших соседей (вычисление эйлерова расстояния, нажоджения всех ближайших соседей и т.д.)
- Разработать функционал для тестирования полученной нейросети на незнакомом ей тексте

Итак приступим

Начнем с датасета. В класс "Детектив" вошли отрывки из произведений Акрура Конан-Дойля, Агаты Кристи, Эдгара По и других писателей детективистов, а в класс "Космическая фантастика" вошли отрывки из произведений Айзека Азимова, Энди Уира, Лоиса Буджолда и других писателей фантастов. Исходные данные представлены в папке "Dataset/Detective" и "Dataset/Space fiction"

А теперь приступим непосредственно к коду

In [6]:
#Импортируем все необходимое
import os
import numpy as np
from math import sqrt

In [18]:
Your_path = "/home/slava/Source"#Можете поменять это для себя
Dir = "/KNN_Text_Class"

source_path = Your_path + Dir + "/Dataset"
stoppath = Your_path + Dir + "/stopwords.txt"

vector_path = Your_path + Dir + "/Vector.txt"
new_vector_path = Your_path + Dir + "/NewVector.txt"

test_path = Your_path + Dir + "/Dataset/Test"

Добавим функционал для подсчета частоты встречи слов в исходном датасете, чтобы определить наиболее часто встречающиеся слова, которые войдут в вектор для дальнейшей классификации. При этом при подстчете слов используется так называемый стоплист - список слов для русского языка, которые довольно часто встречаются в разговорной речи и письме, но при этом не несут никакой смысловой нагрузки, а следовательно не могут быть маркерами для текстов определенного класса. Примеры таких слов приведены в файле stopwords.txt. Эти слова в общем подсчете не участвуют.

In [8]:
def string_norm(str1):
    string = str(str1.lower())

    if string.find('.'):
        string = string.replace('.', '')

    if string.find(','):
        string = string.replace(',', '')

    if string.find(':'):
        string = string.replace(':', '')

    if string.find(';'):
        string = string.replace(';', '')

    if string.find('-'):
        string = string.replace('-', '')

    if string.find('_'):
        string = string.replace('_', '')

    if string.find('!'):
        string = string.replace('!', '')

    if string.find('?'):
        string = string.replace('?', '')

    return string

def word_freq_counter(source_path, stoppath, word):
    filelist = []
    for root, dirs, files in os.walk(source_path + "/" + word): 
        for file in files: 
            #append the file name to the list 
            filelist.append(file)

    #print all the file names 
    for name in filelist: 
        print(name)

    with open(stoppath) as stopfile:
        stoplist = stopfile.read().split()
    #print (stoplist)

    main_word_filelist = []
    main_freq_filelist = []

    print("-----------------")

    for name in filelist: 
        print(name)
        #input()

        with open(source_path + "/" + word + "/" + name) as inputfile:
            list_data = inputfile.read().split() # читаем с файла разбиваем по пробелам
            #print(list_data)
            for item in list_data:
                litem = string_norm(item)
                if litem not in stoplist:
                    #print(litem)
                    #input()
                    if litem in main_word_filelist:
                        litem_index = main_word_filelist.index(litem)
                        litem_freq = main_freq_filelist[litem_index]
                        new_litem_freq = litem_freq + 1
                        main_freq_filelist[litem_index] = new_litem_freq
                    else:
                        main_word_filelist.append(litem)
                        main_freq_filelist.append(1)

    outfile = open(word + ".txt", 'w')

    for i in range(2000):
        max_item_freq = max(main_freq_filelist)
        max_index = main_freq_filelist.index(max_item_freq)
        max_item_word = main_word_filelist[max_index]

        outstring = str(max_item_freq) + ": " + str(max_item_word) + "\n"

        outfile.write(outstring)

        main_word_filelist.pop(max_index)
        main_freq_filelist.pop(max_index)

    outfile.close()
    
def wfc_main(word):
    source_path = "/home/slava/Source/KNN_Text_Class/Dataset"
    stoppath = "/home/slava/Source/KNN_Text_Class/stopwords.txt"
    #word = "Detective"
    #word = "Space_fiction"

    word_freq_counter(source_path, stoppath, word)

In [11]:
# "Detective"
# "Space_fiction"

wfc_main("Detective")
wfc_main("Space_fiction")

8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14
-----------------
8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14
8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14
-----------------
8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14


После того, как мы определили наиболее часто встречающиеся слова для классов Детектив и Космическая фантастика (файлы Detective.txt и Space_fiction.txt), нам предстоит на наше усмотрение выбрать те слова, которые встречаются достаточно часто и по нашему мнению отлично характеризуют данный класс текстов, после чего внести эти слова в наш вектор (файл Vector.txt), а после чего удалить повторяющиеся слова (файл NewVector.txt). Реализуем этот функционал

In [13]:
def repeat_deleter(vector_path, new_vector_path):
    with open(vector_path) as vectorfile:
        vectorlist = vectorfile.read().split()

        rd_vectorlist = list(set(vectorlist))

        ofile = open(new_vector_path, 'w')
        for item in rd_vectorlist:
            ofile.write(item + "\n")

        ofile.close()
        
repeat_deleter(vector_path, new_vector_path)

Далее перейдем к составлению датасета в виде массива для нашего классификатора

Реализуем функционал для составления датасета в виде набора векторов из исходного набора текстов. При этом каждому тексту сопоставляется вектор, который содержит частоты появления каждого из слов, находящихся в нашем контрольном списке слов (файл NewVector.txt)

In [14]:
def normalization_vector(vector):
    norm_vector = [0] * len(vector)#vectorsize
    minv = min(vector)
    maxv = max(vector)
    delta = maxv-minv

    for i in range(len(norm_vector)):
        norm_vector[i] = (vector[i]-minv)/delta

    return norm_vector

def fit_func(full_path_to_text, vectorlist):
    vector = [0] * len(vectorlist)

    with open(full_path_to_text) as inputfile:
        list_data = inputfile.read().split() # читаем с файла разбиваем по пробелам
        for item in list_data:
            litem = string_norm(item)
            if litem in vectorlist:
                index = vectorlist.index(litem)
                v_item = vector[index]
                n_v_item = v_item + 1
                vector[index] = n_v_item

    norm_vector = normalization_vector(vector)
    
    return norm_vector

def dataset_maker(new_vector_path, source_path, word, dataset, number):
    with open(new_vector_path) as vectorfile:
        vectorlist = vectorfile.read().split()

    filelist = []
    for root, dirs, files in os.walk(source_path + "/" + word): 
        for file in files: 
            #append the file name to the list 
            filelist.append(file)

    #print all the file names 
    for name in filelist: 
        print(name)
        
        norm_vector = fit_func(source_path + "/" + word + "/" + name, vectorlist)

        norm_vector.append(number)

        dataset.append(norm_vector)
        # print(norm_vector)
        # input()

    return dataset

def dm_main():
    main_dataset = []

    det = dataset_maker(new_vector_path, source_path, "Detective", main_dataset, 0)
    print("----")
    sf = dataset_maker(new_vector_path, source_path, "Space_fiction", main_dataset, 1)

    #main_dataset = [det, sf]#det is 0, sf is 1
    print(len(main_dataset[0]))

    np.save("DSDataset.npy", main_dataset)

In [15]:
dm_main()

8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14
----
8
12
11
2
18
6
16
7
4
15
19
20
17
10
3
5
13
1
9
14
144


Реализуем функционал для классификации нового текста на основе составленного датасета. При классификации нового текста данный текст прогоняется черех фит-функцию (для него находится вектор частот появления контрольных слов как при составлении датасета из исходных текстов), после чего рассчитывается евклидово расстояние между вектором нового текста и всеми векторами всех текстов, представленных в датасете. Затем определяется принадлежность к классам K ближайших векторов. Класс нового вектора считается равным классу тех векторов, которых больше в выборке из K ближайших векторов

In [24]:
# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors 

# Make a classification prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
    neighbors = get_neighbors(train, test_row, num_neighbors)
    output_values = [row[-1] for row in neighbors]
    prediction = max(set(output_values), key=output_values.count)
    return prediction, output_values

def text_fit(full_path_to_text, full_path_to_vector):
    with open(full_path_to_vector) as inputvec:
        vectorlist = inputvec.read().split() # читаем с файла разбиваем по пробелам
        
    norm_vector = fit_func(full_path_to_text, vectorlist)

    return norm_vector

def p_main():
    dataset = np.load("DSDataset.npy")

    fitd1 = text_fit(test_path + "/" + "D1", new_vector_path)
    fitd2 = text_fit(test_path + "/" + "D2", new_vector_path)
    fits1 = text_fit(test_path + "/" + "S1", new_vector_path)
    fits2 = text_fit(test_path + "/" + "S2", new_vector_path)

    print("0 is Detective, 1 is Space Fiction")
    print("----------------------------------")
    print(" ")

    print("Test text is D1")
    prediction, list_data = predict_classification(dataset, fitd1, 3)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 3, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd1, 5)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 5, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd1, 7)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 7, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd1, 9)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 9, list_data))
    print("----------------------------------------------------")
    print(" ")

    print("Test text is D2")
    prediction, list_data = predict_classification(dataset, fitd2, 3)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 3, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd2, 5)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 5, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd2, 7)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 7, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fitd2, 9)
    print('Expected %d, Got %d, K %d, List %s' % (0, prediction, 9, list_data))
    print("----------------------------------------------------")
    print(" ")

    print("Test text is S1")
    prediction, list_data = predict_classification(dataset, fits1, 3)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 3, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits1, 5)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 5, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits1, 7)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 7, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits1, 9)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 9, list_data))
    print("----------------------------------------------------")
    print(" ")

    print("Test text is S2")
    prediction, list_data = predict_classification(dataset, fits2, 3)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 3, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits2, 5)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 5, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits2, 7)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 7, list_data))
    print("----------------------------------------------------")
    prediction, list_data = predict_classification(dataset, fits2, 9)
    print('Expected %d, Got %d, K %d, List %s' % (1, prediction, 9, list_data))
    print("----------------------------------------------------")

In [25]:
p_main()

0 is Detective, 1 is Space Fiction
----------------------------------
 
Test text is D1
Expected 0, Got 0, K 3, List [0.0, 0.0, 0.0]
----------------------------------------------------
Expected 0, Got 0, K 5, List [0.0, 0.0, 0.0, 0.0, 1.0]
----------------------------------------------------
Expected 0, Got 0, K 7, List [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
----------------------------------------------------
Expected 0, Got 0, K 9, List [0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0]
----------------------------------------------------
 
Test text is D2
Expected 0, Got 0, K 3, List [0.0, 0.0, 0.0]
----------------------------------------------------
Expected 0, Got 0, K 5, List [0.0, 0.0, 0.0, 0.0, 0.0]
----------------------------------------------------
Expected 0, Got 0, K 7, List [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
----------------------------------------------------
Expected 0, Got 0, K 9, List [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0]
-------------------------------------------

Расшифруем вывод:
В первой строке дается обозначение каждого из классов в цифровом виде.
Далее расшифруем вывод на примере одной из строки, в частности:

Expected 0, Got 0, K 5, List [0.0, 0.0, 0.0, 0.0, 1.0]

Expected 0 - ожидалось получить на вход текст из класса 0

Got 0 - текст классифицирован как текст класса 0

K 5 - текст классифицировали по 5 ближайшим соседям

List [0.0, 0.0, 0.0, 0.0, 1.0] - Список ближайших соседей, среди ближайших соседей 4 текста класса 0 и 1 текст класса 1

-----------------------------------

Еще одна строка:

Expected 1, Got 0, K 5, List [1.0, 1.0, 0.0, 0.0, 0.0]

Expected 1 - ожидалось получить на вход текст из класса 1

Got 0 - текст классифицирован как текст класса 0, мы потерпели неудачу :(

K 5 - текст классифицировали по 5 ближайшим соседям

List [1.0, 1.0, 0.0, 0.0, 0.0] - Список ближайших соседей, среди ближайших соседей 3 текста класса 0 и 2 текст класса 1